### Kernel Patterns


In [ ]:
# ElementWiseKernel

!pip install pycuda

import pycuda.autoinit
from   pycuda import gpuarray
from   pycuda.elementwise import ElementwiseKernel
import numpy as np
import time


prod_kernel = ElementwiseKernel (
    arguments = "float *a, float *b, float *c",
    operation = "c[i] = a[i] * b[i];",
    name      = "prod_kernel"
)


if __name__ == '__main__':

    size = 1024

    h_a = np.random.rand(size * size).reshape(size, size).astype(np.float32)
    h_b = np.random.rand(size * size).reshape(size, size).astype(np.float32)

    print("Matrix A: ", h_a)
    print("Matrix B: ", h_b)

    start = time.time()
    d_a = gpuarray.to_gpu(h_a)
    d_b = gpuarray.to_gpu(h_b)
    d_c = gpuarray.empty_like(d_a)
    
    prod_kernel(d_a, d_b, d_c)

    h_c = d_c.get()
    end = time.time()
    gpu_time = end - start
    print("GPU time: %.6f" % (gpu_time))
    print("Matrix: ", h_c)

In [ ]:
# Mandelbrot

!pip install pycuda


import pycuda.driver as CUDA
import pycuda.autoinit
from   pycuda import gpuarray
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.use('Agg')
import pandas as pd
import time

from pycuda.elementwise import ElementwiseKernel

#  Poniendo w y h a 512 los tiempos son aprox:
#    CPU ~ 36 secs
#    GPU ~ 0.2 secs

def cpu_mandelbrot(w, h, r_low, r_high, i_low, i_high, max_iters, upper_bound):

    real_vals = np.linspace(r_low, r_high, w)
    imag_vals = np.linspace(i_low, i_high, h)

    # we will represent members as 1, non-members as 0.

    mandelbrot_graph = np.ones((h,w), dtype=np.float32)

    for x in range(w):

        for y in range(h):

            c = np.complex64( real_vals[x] + imag_vals[y] * 1j  )
            z = np.complex64(0)

            for i in range(max_iters):

                z = z**2 + c

                if(np.abs(z) > upper_bound):
                    mandelbrot_graph[y,x] = 0
                    break

    return mandelbrot_graph



mandel_ker = ElementwiseKernel(
"pycuda::complex<float> *lattice, float *mandelbrot_graph, int max_iters, float upper_bound",

"""
mandelbrot_graph[i] = 1;
pycuda::complex<float> c = lattice[i];
pycuda::complex<float> z(0,0);

for (int j = 0; j < max_iters; j++) {
    z = (z * z) + c;
    if(abs(z) > upper_bound) {
        mandelbrot_graph[i] = 0;
        break;
    }
}
""",

"mandel_ker")



def mandelbrot (w, h, r_low, r_high, i_low, i_high, max_iters, upper_bound):
    real_values = np.matrix( np.linspace(r_low,  r_high, w, dtype=np.complex64))
    img_values  = np.matrix( np.linspace(i_high, i_low,  h, dtype=np.complex64)) * 1j

    m_lattice = np.array(real_values + img_values.transpose(), dtype=np.complex64)
    # print("Lattice:", m_lattice.shape)
    # print(m_lattice)
    # copy complex lattice to the GPU
    m_lattice_gpu = gpuarray.to_gpu(m_lattice)
    # allocate an empty array on the GPU
    m_graph_gpu = gpuarray.empty(shape=m_lattice.shape, dtype=np.float32)
    # Launch kernel
    mandel_ker(m_lattice_gpu, m_graph_gpu, np.int32(max_iters), np.float32(upper_bound))
    # GPU to host memoory
    m_graph = m_graph_gpu.get()
    return m_graph


if __name__ == '__main__':

    start = time.time()
    m = cpu_mandelbrot(w =256, h = 256,
                   r_low = -2, r_high = 2,
                   i_low = -2, i_high = 2,
                   max_iters = 500,
                   upper_bound = 2.5)
    end = time.time()
    cpu_time = end - start
    print("CPU time:", cpu_time)

    start = time.time()
    m_gpu = mandelbrot(w =256, h = 256,
                   r_low = -2, r_high = 2,
                   i_low = -2, i_high = 2,
                   max_iters = 500,
                   upper_bound = 2.5)
    end = time.time()
    gpu_time = end - start
    print("GPU time:", gpu_time)


In [ ]:
# Parallel Scan kernel:

!pip install pycuda

import pycuda.autoinit
from   pycuda import gpuarray
import numpy as np
from pycuda.scan import InclusiveScanKernel, ExclusiveScanKernel

if __name__ == '__main__':
    
    cumsum_i = InclusiveScanKernel(
        dtype       = np.float32, 
        neutral     = "0",
        scan_expr   = "a+b", 
        name_prefix = "cumsum"
    )
    
    cumsum_e = ExclusiveScanKernel(
        dtype       = np.float32, 
        neutral     = "0",
        scan_expr   = "a+b", 
        name_prefix = "cumsum"
    )

    h_i = np.arange(10, dtype=np.float32)
    d_i = gpuarray.to_gpu(h_i)    
    cumsum_i(d_i)
    h_i = d_i.get()
    print("Resultado (inclusive): ", h_i)

    h_e = np.arange(10, dtype=np.float32)
    d_e = gpuarray.to_gpu(h_e)    
    cumsum_e(d_e)
    h_e = d_e.get()
    print("Resultado (exclusive) ", h_e)



In [ ]:
# Otros patrones de kernel: 
!pip install pycuda

import pycuda.autoinit
from   pycuda import gpuarray
import numpy as np
from pycuda.reduction import ReductionKernel

if __name__ == '__main__':
    
    dot = ReductionKernel(        
        arguments   = "const float *x, const float *y",
        map_expr    = "x[i] * y[i]", 
        reduce_expr = "a+b", 
        neutral     = "0",
        dtype_out   = np.float32
    )
    
    h_a = np.arange(10, dtype=np.float32)
    h_b = np.arange(10, dtype=np.float32)

    d_a = gpuarray.to_gpu(h_a)
    d_b = gpuarray.to_gpu(h_b)

    d_c = dot(d_a, d_b)

    h_c = d_c.get()
    print("Resultado: ", h_c)
